In [5]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import glob
import pathlib

In [6]:
# 获取所有图片路径
train_img_path = glob.glob('./dataset/dc_2000/train/*/*')

In [7]:
# 获取对应图片的label
label_names = os.listdir('./dataset/dc_2000/train/')
label_to_index = dict((name, index) for index, name in enumerate(label_names))
train_img_label = [label_to_index[pathlib.Path(p).parent.name] for p in train_img_path]

In [14]:
def load_preprocess_img(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [360, 360])
    # 对图片进行随机裁剪
    image = tf.image.random_crop(image, [256, 256, 3])
    # 随机进行翻转
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    # 随机增加亮度Å
    image = tf.image.random_brightness(image, 0.5)
    # 增加对比度
    image = tf.image.random_contrast(image, 0, 1)
    image = tf.cast(image, tf.float32)
    image = image/255
    label = tf.reshape(label, [1])
    return (image, label)

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [10]:
train_image_ds = tf.data.Dataset.from_tensor_slices((train_img_path, train_img_label))
train_image_ds = train_image_ds.map(load_preprocess_img, num_parallel_calls=AUTOTUNE)

In [13]:
BATCH_SIZE = 32

In [44]:
train_image_ds = train_image_ds.shuffle(2000).batch(BATCH_SIZE)
train_image_ds = train_image_ds.prefetch(AUTOTUNE)

In [48]:
# 这里可以通过生成器来读取我们的数据
imgs, labels = next(iter(train_image_ds))
imgs.shape

TensorShape([32, 256, 256, 3])

In [52]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(256, 256, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(128, (3,3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(256, (3,3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Conv2D(512, (3,3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(1))

In [53]:
# 定义损失函数
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [54]:
# 定义优化器
optimizer = tf.keras.optimizers.Adam()

In [57]:
# 定义评估器
m_metrics = tf.keras.metrics.Mean()
a_metrics = tf.keras.metrics.Accuracy()

In [69]:
# 定义每一步的函数
def train_step(model, images, label):
    with tf.GradientTape() as tape:
        pred = model(images)
        loss_step = loss(label, pred)
    grads = tape.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    m_metrics(loss_step)
    a_metrics(label, tf.cast(pred>0, tf.int32))

In [70]:
train_loss_result = []
train_acc_result = []

In [ ]:
for epoch in range(30):
    for images_, label_ in train_image_ds:
        train_step(model, images_, label_)
        print('.', end='')
    print()
    train_loss_result.append(m_metrics.result())
    train_acc_result.append(a_metrics.result())
    
    print("epoch: {}, loss: {:.3f}, acc: {:.3f}".format(
        epoch + 1,
        m_metrics.result(), 
        a_metrics.result()
    ))
    
    m_metrics.reset_states()
    a_metrics.reset_states()